In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time, os
from datetime import datetime
import pandas as pd

In [2]:
# review link link
link = "https://play.google.com/store/apps/details?id=com.kbstar.kbpfm&showAllReviews=true"

In [3]:
# how many scrolls we need
scroll_cnt = 4

In [4]:
driver = webdriver.Chrome("C:/chrome/chromedriver.exe")
driver.get(link)

In [5]:
os.makedirs('result', exist_ok=True)

In [6]:
while True:
    # (1) 4번의  스크롤링
    for i in range(4):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    # (2) 더보기 클릭
    try:
        driver.find_element_by_xpath("//span[@class='RveJvd snByac']").click()
    except:
        print('끝')
        break

끝


In [7]:
# get review containers
reviews = driver.find_elements_by_xpath('//*[@jsname="fk8dgd"]//div[@class="d15Mdf bAhLNe"]')

In [8]:
print('There are %d reviews avaliable!' % len(reviews))
print('Writing the data...')

There are 614 reviews avaliable!
Writing the data...


In [9]:
# create empty dataframe to store data
df = pd.DataFrame(columns=['date', 'ratings', 'helpful', 'comment'])

In [10]:
# get review data
for review in reviews:
    # parse string to html using bs4
    soup = BeautifulSoup(review.get_attribute('innerHTML'), 'html.parser')
    
    # review date
    date = soup.find(class_='p2TkOb').text
    date = datetime.strptime(date, '%Y년 %m월 %d일')
    date = date.strftime('%Y-%m-%d')
    
    # ratiabsng
    ratings = int(soup.find('div', role='img').get('aria-label').replace('별표 5개 만점에', '').replace('개를 받았습니다.', '').strip())

    # helpful
    helpful = soup.find(class_='jUL89d y92BAb').text
    if not helpful:
        helpful = 0
  
    # review text
    comment = soup.find('span', jsname='fbQN7e').text
    if not comment:
        comment = soup.find('span', jsname='bN97Pc').text

    # append to dataframe
    df = df.append({
        'date': date,
        'ratings': ratings,
        'helpful': helpful,
        'comment': comment
    }, ignore_index=True)

In [11]:
# finally save the dataframe into csv file
df.to_csv('마이머니_Playstore.csv', encoding='utf-8-sig', index=False)
driver.stop_client()
driver.close()

print('Done!')

Done!


In [12]:
train = pd.read_csv('마이머니_Playstore.csv')
train

,date,ratings,helpful,comment
0,2020-11-06,3,13,솔직히 많이 부족한 앱이라고 생각합니다. 여러 은행 등록가능한건 좋지만 이런 어플에...
1,2020-11-15,3,4,데이터가 중복으로 떠서 지우기를 반복하다가 서비스 해지까지했는데 업데이트가 있었네요...
2,2020-11-01,2,17,"기본 테스트도 안하고 출시한 앱 같아요. 인증서 등록 무한반복, 지문등록 무한반복 ..."
3,2020-10-29,4,16,KB마이머니에 로그인 하면 KB스타알림에 스마트폰뱅킹 아이디로 로그인 했다고 알림이...
4,2020-09-01,1,81,"가계부는 매일 입력된 지출카테고리를, 매달 지출 카테고리별 총 금액으로, 한눈에 보..."
...,...,...,...,...
609,2018-05-05,2,0,안좋아
610,2017-04-23,5,0,좋아요
611,2017-05-11,5,1,좋아요
612,2020-06-13,5,0,굿
